# **Instituto Tecnológico y de Estudios superiores de Monterrey**

## Escuela de Ingeniería y Ciencias

### Maestría en inteligencia artificial aplicada

#### Proyecto integrador

*Avance 3*

Alumnos:  
Alfonso de Lucas Espinosa - A01795118  
Pablo Andrés Estrada Flores - A01795212


#### Desarrollo

**1. ¿Qué algoritmo se puede utilizar como baseline para predecir las variables objetivo?**

El algoritmo seleccionado como punto de partida  baseline para predecir las letras de la Lengua de Señas Mexicana (LSM) es una Red Neuronal Convolucional (CNN). La idoneidad de este algoritmo se fundamenta en varios aspectos clave del problema y los datos:

1.1 Tipo y Estructura de los Datos:

  Los datos de entrada consisten en las coordenadas (x, y, z) de 21 puntos de referencia (landmarks) de la mano, extraídos mediante MediaPipe.


1.2 Cantidad de Datos:

  El conjunto de datos es considerable. Por ejemplo, el conjunto de prueba contiene 55,231 muestras. Esta magnitud es adecuada para entrenar una CNN de manera efectiva, permitiéndole aprender patrones complejos sin un sobreajuste inmediato, especialmente con la división adecuada en conjuntos de entrenamiento, validación y prueba (60%/20%/20% respectivamente).

1.3 Aprendizaje Automático de Características Relevantes:

  Las CNN son capaces de aprender jerarquías de características espaciales directamente de los datos crudos o mínimamente procesados. En este caso, pueden identificar patrones relevantes en la disposición tridimensional de los landmarks de la mano para cada seña. Esta capacidad reduce la necesidad de ingeniería de características manual (como ángulos específicos o distancias precalculadas).

1.4 Rendimiento en Tareas Similares:

  La elección se respalda en el desempeño significativamente superior que las CNN han demostrado en tareas similares de clasificación de lenguas de señas, particularmente cuando se utiliza MediaPipe para la extracción de información de la mano.

1.5 Interpretabilidad vs. Precisión:

  Si bien la interpretabilidad directa de las características aprendidas por las capas convolucionales puede ser compleja, el objetivo principal en este contexto es la alta precisión en la clasificación de las señas. El modelo CNN propuesto, está diseñado para maximizar esta precisión.
  Por estas razones, una CNN se considera un baseline fuerte y apropiado, ya que está bien adaptada a la naturaleza espacial y estructurada de los datos de landmarks de la mano y ha demostrado éxito en dominios análogos.

**2. ¿Se puede determinar la importancia de las características para el modelo generado?**

Las características geométricas han probado ser un apoyo clave en el desarrollo de modelos tradicionales de aprendizaje automático, como lo demuestra el trabajo de Anzar, H. (2022), donde se utiliza una combinación de estructuras K-ary tree y hashing para la identificación de gestos de la mano.

No obstante, dichas características pueden afectar negativamente el desempeño de una CNN, ya que este tipo de red es capaz de aprender patrones espaciales de forma automática, requiriendo únicamente datos estructurados y, como mucho, normalizados para esta aplicación.

Procedemos entonces con la generación de nuestro modelo CNN básico utilizando la librería PyTorch.

#### Carga de paqueterías

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report

#### Carga el procesamiento de datos; separación en conjuntos

In [3]:
# Función para leer y preprocesar CSV
def prepare_for_cnn_from_csv(path_csv):
    df = pd.read_csv(path_csv)
    X = df.drop(columns=['label']).values.reshape(-1, 21, 3)
    y = df['label'].values

    # Normalización espacial
    wrist = X[:, 0:1, :]
    X -= wrist
    distances = np.linalg.norm(X[:, 1:, :], axis=2)
    scale = np.mean(distances, axis=1, keepdims=True) + 1e-6
    X[:, 1:, :] /= scale[:, np.newaxis]

    # Codificar etiquetas
    encoder = LabelEncoder()
    y_encoded = encoder.fit_transform(y)

    return X.astype(np.float32), y_encoded.astype(np.longlong), encoder

# Ruta local del CSV
csv_path = "/content/drive/MyDrive/Colab Notebooks/LSM/03avance/hand_landmarks_raw.csv"
X, y, label_encoder = prepare_for_cnn_from_csv(csv_path)

# Separar en train/val/test (60% train, 20% val, 20% test)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=42)

#### Adaptación del dataset a PyTorch

In [4]:
class HandLandmarksDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X)
        self.y = torch.tensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = HandLandmarksDataset(X_train, y_train)
val_dataset = HandLandmarksDataset(X_val, y_val)
test_dataset = HandLandmarksDataset(X_test, y_test)

#### Carga de datos

In [5]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

#### Definición del modelo

El modelo obtiene un tensor de forma (32, 21, 3), donde 32 es el tamaño del batch definido en el data loader, 21 es el número de puntos de la mano y 3 son las dimensiones de cada punto (x, y, z).
Tiene dos capas Convid1d que ocupan la función de activación ReLU  seguidas de AdaptiveMaxPool que dan salida a un vector de logits de tamaño igual al número de letras (clases))

In [6]:
class LandmarkCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.fc1 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # (batch, 3, 21)
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.pool(x).squeeze(-1)
        return self.fc1(x)

#### Entrenamiento

Se entrena el modelo y se evaluan los conjuntos de train y val, para valorar un posible sub/sobre ajuste inicial. Se realiza en 10 epochs.

In [7]:
model = LandmarkCNN(num_classes=len(label_encoder.classes_))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Evaluación en entrenamiento
    model.eval()
    train_correct = 0
    train_total = 0
    with torch.no_grad():
        for batch_X, batch_y in train_loader:
            outputs = model(batch_X)
            preds = torch.argmax(outputs, dim=1)
            train_correct += (preds == batch_y).sum().item()
            train_total += batch_y.size(0)
    train_acc = train_correct / train_total

    # Evaluación en validación
    val_correct = 0
    val_total = 0
    val_loss = 0.0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            preds = torch.argmax(outputs, dim=1)
            val_correct += (preds == batch_y).sum().item()
            val_total += batch_y.size(0)
            val_loss += loss.item()
    val_acc = val_correct / val_total
    val_loss_avg = val_loss / len(val_loader)

    print(f"Epoch {epoch+1}/{epochs} - Train Acc: {train_acc:.4f} - Val Acc: {val_acc:.4f} - Val Loss: {val_loss_avg:.4f}")

Epoch 1/10 - Train Acc: 0.9199 - Val Acc: 0.9172 - Val Loss: 0.2607
Epoch 2/10 - Train Acc: 0.9412 - Val Acc: 0.9387 - Val Loss: 0.1819
Epoch 3/10 - Train Acc: 0.9554 - Val Acc: 0.9527 - Val Loss: 0.1447
Epoch 4/10 - Train Acc: 0.9664 - Val Acc: 0.9650 - Val Loss: 0.1043
Epoch 5/10 - Train Acc: 0.9728 - Val Acc: 0.9699 - Val Loss: 0.0941
Epoch 6/10 - Train Acc: 0.9733 - Val Acc: 0.9710 - Val Loss: 0.0902
Epoch 7/10 - Train Acc: 0.9709 - Val Acc: 0.9692 - Val Loss: 0.0928
Epoch 8/10 - Train Acc: 0.9773 - Val Acc: 0.9752 - Val Loss: 0.0742
Epoch 9/10 - Train Acc: 0.9757 - Val Acc: 0.9737 - Val Loss: 0.0746
Epoch 10/10 - Train Acc: 0.9810 - Val Acc: 0.9778 - Val Loss: 0.0662


#### Evaluación

Se evalúa el conjunto test

In [8]:
model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(batch_y.cpu().numpy())

print("\nEvaluación en el conjunto de prueba:")
target_names = label_encoder.classes_
report = classification_report(all_targets, all_preds, target_names=target_names)
print(report)


Evaluación en el conjunto de prueba:
              precision    recall  f1-score   support

           A       0.98      1.00      0.99      2764
           B       1.00      0.99      0.99      2733
           C       1.00      1.00      1.00      2578
           D       0.99      0.96      0.98      2692
           E       0.97      0.99      0.98      2692
           F       0.98      0.99      0.99      2713
           G       0.99      1.00      0.99      2595
           H       1.00      0.99      0.99      2455
           I       1.00      0.98      0.99      2696
           L       1.00      0.99      0.99      2685
           M       0.96      0.95      0.96      2601
           N       0.95      0.96      0.96      2444
           O       0.99      0.97      0.98      2612
           P       0.98      0.99      0.98      2607
           R       0.87      0.99      0.93      2659
           S       0.99      0.99      0.99      2599
           T       0.98      0.99      0.99

**3. ¿El modelo está sub/sobreajustando los datos de entrenamiento?**

  Para determinar si el modelo CNN está subajustando o sobreajustando los datos de entrenamiento, se analizaron las métricas de evaluación en los conjuntos de entrenamiento y validación durante las épocas de entrenamiento, así como el rendimiento final en el conjunto de prueba.
  
  Durante las 10 épocas de entrenamiento, se monitorearon la precisión en el entrenamiento (Train Acc), la precisión en la validación (Val Acc) y la pérdida en la validación (Val Loss).

  Se observa que la precisión en el conjunto de entrenamiento y en el de validación son consistentemente cercanas a lo largo de las épocas, aumentando ambas de manera estable.

  La pérdida de validación (Val Loss) disminuye consistentemente desde 0.2607 en la primera época hasta 0.0662 en la décima época, lo que indica que el modelo está generalizando bien a datos no vistos durante el entrenamiento.

  El modelo alcanzó una precisión (accuracy) global del 98% en el conjunto de prueba. Este valor es muy cercano a la precisión de entrenamiento (98.10% en la última época) y a la precisión de validación (97.78% en la última época).

* La pequeña diferencia entre las precisiones de entrenamiento, validación y prueba sugiere que el modelo no se ha memorizado los datos de entrenamiento a expensas de la generalización. La pérdida de validación no muestra una tendencia a aumentar, por lo cual no está sobreajuste.
* Las altas precisiones obtenidas en todos los conjuntos (superiores al 97%) indican que el modelo tiene la capacidad suficiente para aprender la tarea de clasificación de manera efectiva y no está subajustado.

**4. ¿Cuál es la métrica adecuada para este problema de negocio?**

El problema de negocio consiste en la clasificación precisa de gestos estáticos de la Lengua de Señas Mexicana (LSM) a partir de datos de landmarks de la mano. El objetivo es desarrollar un sistema que pueda interpretar estas señas de manera fiable.

El conjunto de métricas empleadas (Accuracy, Precision, Recall, F1-Score por clase, Macro F1 y Weighted F1) es exhaustivo y adecuado para este problema. Los valores consistentemente altos obtenidos demuestran un rendimiento sobresaliente y robusto del modelo empleado (CNN), tanto a nivel general como para cada letra individual.

* Accuracy Global: Con 97%, es una métrica clave que muestra el éxito general.

* F1-Score (Macro y Ponderado): Con valores de 0.98 para ambos, indican un rendimiento excelente y balanceado entre las clases.

* Precision y Recall por Clase: Todas son muy altas, lo que significa que el modelo es fiable para cada letra individualmente.



**5. ¿Cuál debería ser el desempeño mínimo a obtener?**

El modelo CNN ha superado con creces tanto el desempeño mínimo esperado (>85%) y además el objetivo final del proyecto (>90%), alcanzando un accuracy en el conjunto de prueba del 98%. Este resultado no solo valida la efectividad del conjunto de características y la arquitectura del modelo para este dataset, sino que también establece un estándar alto de rendimiento para el proyecto.

#### Conclusiones



1.   El modelo de Red Neuronal Convolucional (CNN) implementado ha demostrado una alta efectividad para la clasificación de gestos estáticos de la Lengua de Señas Mexicana a partir de datos de puntos de referencia de la mano. Este enfoque se alinea con hallazgos previos que destacan la idoneidad de las CNN para tareas de reconocimiento de patrones espaciales en datos de señas.


2. Los resultados de la evaluación indican que el modelo entrenado generaliza adecuadamente los datos. Las métricas de desempeño en el conjunto de validación durante el entrenamiento mostraron una convergencia estable, con una pérdida de validación que disminuyó consistentemente hasta las últimas épocas (Val Loss: 0.0746 en la época 9). Adicionalmente, la exactitud (accuracy) obtenida en el conjunto de prueba, que alcanzó un 0.98, junto con altos valores de precisión, recall y F1-score para la mayoría de las clases respalda la capacidad del modelo para realizar predicciones fiables.

3. El desempeño alcanzado, con una exactitud global del 98% en el conjunto de prueba, posiciona al modelo como una base sólida para futuras investigaciones y potenciales aplicaciones en sistemas de traducción o asistencia para la Lengua de Señas Mexicana.




#### Referencias

- Ansar, H., Ksibi, A., Jalal, A., Shorfuzzaman, M., Alsufyani, A., Alsuhibany, S. A., & Park, J. (2022). Dynamic hand gesture recognition for smart lifecare routines via K-ary tree hashing classifier. Applied Sciences, 12(13), 6481.
- Zhou, Y., Zhao, K., Jin, L., & Zhan, C. (2024). End-to-End Video-Based Sign Language Recognition with CNN-Transformer Encoder-Decoder. arXiv.
- Ramírez Sánchez, J. E., Anguiano Rodríguez, A., & González Mendoza, M. (2021). Interpretación en tiempo real de lengua de señas mexicana con CNN y HMM. Research in Computing Science, 150(6), 91–100.